In [118]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [133]:
#使用pickle模块储存决策树,pickle模块在Python3中不用下载但在Python2中不同
def storeTree(inputTree,filename):
    import pickle
    fw = open(filename,'w')
    pickle.dump(inputTree,fw)
    fw.close()
def grabTree(filename):
    import pickle
    fr = open(filename)
    return pickle.load(fr)

In [119]:
def calcShannonEnt(label):
    label = np.array(label)
    uni_value = np.unique(label)
    HD = 0
    for uni in uni_value:
        Pi = len(label[label==uni])/len(label)
        HD += - Pi* math.log2(Pi)
    return HD

In [120]:
def splitDataSet(data,label,j,value):
    retDataSet = []
    retLabelSet = []
    for i in range(len(data)):
        if data[i][j] == value:
            retDataSet.append(data[i][:j]+data[i][j+1:])
            retLabelSet.append(label[i])
    return retDataSet,retLabelSet

In [121]:
def major(label):
    #若是不使用numpy来找label中为unique的值会出错
    label = np.array(label)
    unique_label = set(label)
    times = {}
    for unique in unique_label:
        times[unique] = len(label[label==unique])
    return max(times,key = times.get)

In [122]:
def majorClass(data,label,col_name):
    leaf_node = {col_name[0]:{}}
    featVector = np.array(data)[:,0]
    feat = set(featVector)
    for f in feat:
        clas = label[featVector==f]
        leaf_node[col_name[0]][f] = major(clas)
    return leaf_node

In [123]:
def chooseBestFeature(data,label):
    data = np.array(data)
    label = np.array(label)
    bestFeature = 0
    bestInfoGain = 0
    H_D = calcShannonEnt(label)
    for i in range(data.shape[1]):
        H_D_A = 0
        featureVector = data[:,i]
        feature = set(featureVector)
        for f in feature:
            prob = len(data[featureVector==f])/len(data)
            H_D_A += prob*calcShannonEnt(label[featureVector==f])
        infoGain = H_D - H_D_A
        if infoGain > bestInfoGain:
            bestFeature = i
            bestInfoGain = infoGain
    return bestFeature

In [130]:
def createTree(data,label,col_name):
    label = np.array(label)
    unique_label = np.unique(label)
    if len(unique_label) == 1:
        return unique_label[0]
    if len(col_name) == 1:
        return majorClass(data,label,col_name)
    bestFeature = chooseBestFeature(data,label)
    featureVector = np.array(data)[:,bestFeature]
    colsname = col_name[:]         #在这里需对col_name复制，否则在原本的col_name上操作
    col = colsname[bestFeature]
    del(colsname[bestFeature])
    features = set(featureVector)
    tree= {col:{}}
    for feat in features:
        splitData = splitDataSet(data,label,bestFeature,feat)[0]
        splitLabel = splitDataSet(data,label,bestFeature,feat)[1]
        tree[col][feat] = createTree(splitData,splitLabel,colsname) 
    return tree

In [131]:
def classify(Tree,data,col_name):
    key = list(Tree.keys())[0]
    findDict = Tree[key]
    index = col_name.index(key)
    nextDict = findDict[data[index]]
    if type(nextDict).__name__ == 'dict':
        className = classify(nextDict,data,col_name)
    else:
        className = nextDict
    return className

In [1]:
fileName = 'C:/Users/Administrator/Desktop/机器学习实战代码/MLiA_SourceCode/Ch03/lenses.txt'
file = open(fileName)
data = []
label = []
for context in file.readlines():
    cureline = context.split()
    data.append([num for num in cureline[:4]])
    if len(cureline) == 5 :
        label.append(cureline[4])
    else:
        label.append(cureline[4]+' '+cureline[5])
data,label

([['young', 'myope', 'no', 'reduced'],
  ['young', 'myope', 'no', 'normal'],
  ['young', 'myope', 'yes', 'reduced'],
  ['young', 'myope', 'yes', 'normal'],
  ['young', 'hyper', 'no', 'reduced'],
  ['young', 'hyper', 'no', 'normal'],
  ['young', 'hyper', 'yes', 'reduced'],
  ['young', 'hyper', 'yes', 'normal'],
  ['pre', 'myope', 'no', 'reduced'],
  ['pre', 'myope', 'no', 'normal'],
  ['pre', 'myope', 'yes', 'reduced'],
  ['pre', 'myope', 'yes', 'normal'],
  ['pre', 'hyper', 'no', 'reduced'],
  ['pre', 'hyper', 'no', 'normal'],
  ['pre', 'hyper', 'yes', 'reduced'],
  ['pre', 'hyper', 'yes', 'normal'],
  ['presbyopic', 'myope', 'no', 'reduced'],
  ['presbyopic', 'myope', 'no', 'normal'],
  ['presbyopic', 'myope', 'yes', 'reduced'],
  ['presbyopic', 'myope', 'yes', 'normal'],
  ['presbyopic', 'hyper', 'no', 'reduced'],
  ['presbyopic', 'hyper', 'no', 'normal'],
  ['presbyopic', 'hyper', 'yes', 'reduced'],
  ['presbyopic', 'hyper', 'yes', 'normal']],
 ['no lenses',
  'soft',
  'no lenses',

In [156]:
colSname = ['age','prescript','astigmatic','tearRate']
eye_tree = createTree(data,label,colSname)
eye_tree

{'tearRate': {'reduced': 'no lenses',
  'normal': {'astigmatic': {'yes': {'prescript': {'myope': 'hard',
      'hyper': {'age': {'young': 'hard',
        'presbyopic': 'no lenses',
        'pre': 'no lenses'}}}},
    'no': {'age': {'young': 'soft',
      'presbyopic': {'prescript': {'myope': 'no lenses', 'hyper': 'soft'}},
      'pre': 'soft'}}}}}}